In [ ]:
#Here we are trying to cheat on model, By assking it predict base on info we are hiding from it!

#let's call our data ( Assume its name is data.csv ) and create a copy of some of its rows!

import pandas as pd

df = pd.read_csv('input/train/data.csv')

df.shape

In [ ]:
df_pred = df.iloc[3600:,:]
df = df.iloc[:3600,:]
df_pred.shape, df.shape

In [ ]:
df.to_csv('input/train/data-rowcut.csv', index=False)
df_pred.to_csv('input/test/data-pred.csv', index=False)
print("Now We Have Two Data set Which one of them Is not being seen by Model During trian and test")

In [ ]:
# First let us assume that you are used the enhanced version which saved args to a pickle file:
# We are loading the args same to when trained the model:
# It tipically save it in input folder:
import pickle

with open('input/args.pkl', 'rb') as f:
    loaded_arg = pickle.load(f)

print("Args object loaded from args.pkl")
print(loaded_arg)

In [ ]:
#Just a few changes!
loaded_arg.test_size = None
loaded_arg.max_use_of_row = 'All Except 3 Days'
loaded_arg.pred_data_path = 'data-pred.csv'
loaded_arg.is_training = 0
print(loaded_arg)

In [ ]:
# We should use Model class :
# let us import same as when we trained model:
from utils.tools import dotdict
import torch
from experiments.exp_long_term_forecasting import Exp_Long_Term_Forecast
from experiments.exp_long_term_forecasting_partial import Exp_Long_Term_Forecast_Partial
import random
import numpy as np

if loaded_arg.exp_name == 'partial_train':
    Exp = Exp_Long_Term_Forecast_Partial
else:
    Exp = Exp_Long_Term_Forecast

In [ ]:
exp = Exp(loaded_arg)

In [ ]:
#setting is same as the name of the folder that is saved our model:
setting = 'test_iTransformer_custom_MS_ft15_sl1_ll3_pl2_dm1_nh1_el1_dl2_df1_fctimeF_ebTrue_dttest_projection_0'

In [ ]:
exp.predict(setting, load=True)

In [ ]:
import os
predicted_path = os.path.join('results', setting, 'Preds real_prediction.npy')
predicted_values = np.load(predicted_path)
predicted_close = predicted_values[0,:,-1].reshape(-1,1)
predicted_close

In [ ]:
#This is the last 3 days of the pred dataset :
import pandas as pd
df = pd.read_csv('input/test/data-pred.csv')
df['Close'].tail(4)

In [ ]:
import joblib
scaler = joblib.load('input/test/scaler.pkl')
iversed_close = scaler.inverse_transform(predicted_close)
[round(any_) for any_ in iversed_close.reshape(-1).tolist()]

In [ ]:
# مدل در هنگام پیش بینی ۱ ردیف آخر را فقط دریافت می کند
# از آنجا که ما با آرگومان مکس ردیف یوز ، برای او مشخص کردیم که ۳ تای آخر را دست نزند
# پس چهارمین مانده به آخر را گرفته است
#برای اینکه از اسکیل مطمئن باشیم آن را بر میگردانیم تا یکسان باشند

scaler.inverse_transform(exp.batch_y[0,:,-1].reshape(-1,1))

In [ ]:
# OR OR ##

In [ ]:
# OR IF YOU HAVE NOT YOUR ARGS ! HERE LETS TRAIN FROM SCRATCH ! #

In [ ]:
# Define the arg object
arg = dotdict()

arg.root_path = 'input/train'
arg.data_path = 'data-rowcut.csv'
arg.pred_root_path = 'input/test'
arg.pred_data_path = 'data-pred.csv'

In [ ]:
from utils.tools import dotdict
import torch
from experiments.exp_long_term_forecasting import Exp_Long_Term_Forecast
from experiments.exp_long_term_forecasting_partial import Exp_Long_Term_Forecast_Partial
import random
import numpy as np

# Setting the seed
fix_seed = 2023
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


# NEW OPTIONS : #
arg.scale = True
arg.test_size = 0.2
arg.kind_of_scaler = 'Standard'
arg.name_of_col_with_date = 'date'
arg.kind_of_optim = 'default'
arg.criter = 'default'
arg.do_visual = False
arg.max_use_of_row = 'All Except 3 Days'#It also can be 'All Except a Week' or 'No Lim'
#        #      #

arg.is_training = 1
arg.model_id = 'test'
arg.model = 'iTransformer'
arg.data = 'custom'
arg.features = 'MS'
arg.target = 'Close'
arg.freq = 'b'
arg.checkpoints = './checkpoints/'
arg.seq_len = 1*5*3
arg.label_len = 1*1
arg.pred_len = 1*3
arg.enc_in = 6
arg.dec_in = 6
arg.c_out = 1
arg.d_model = 2
arg.n_heads = 1
arg.e_layers = 1
arg.d_layers = 1
arg.d_ff = 2
arg.moving_avg = 25
arg.factor = 1
arg.distil = True
arg.dropout = 0.01
arg.embed = 'timeF'
arg.activation = 'ReLU'
arg.num_workers = 1
arg.itr = 1
arg.train_epochs = 3
arg.batch_size = 64
arg.patience = 10
arg.learning_rate = 0.9
arg.des = 'test'
arg.loss = 'MSE'
arg.lradj = 'type1'
arg.use_amp = False
arg.use_gpu = True if torch.cuda.is_available() else False
arg.gpu = 0
arg.use_multi_gpu = False
arg.devices = '0,1,2,3'
arg.exp_name = 'MTSF'
arg.channel_independence = False
arg.inverse = False
arg.class_strategy = 'projection'
arg.efficient_training = False
arg.use_norm = True
arg.partial_start_index = 0

print('Args in experiment:')
print(arg)

if arg.exp_name == 'partial_train':
    Exp = Exp_Long_Term_Forecast_Partial
else:
    Exp = Exp_Long_Term_Forecast


In [ ]:
if arg.is_training:
    for ii in range(arg.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
                arg.model_id,
                arg.model,
                arg.data,
                arg.features,
                arg.seq_len,
                arg.label_len,
                arg.pred_len,
                arg.d_model,
                arg.n_heads,
                arg.e_layers,
                arg.d_layers,
                arg.d_ff,
                arg.factor,
                arg.embed,
                arg.distil,
                arg.des,
                arg.class_strategy, ii)
        
        exp = Exp(arg)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)
        
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        
        train_losses = exp.train_losses##### --->>> Use These To Plot the Loss Values
        test_losses = exp.test_losses####   --->>> Use These To Plot the Loss Values
        
        exp.test(setting)
        
        if arg.do_predict:
            print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.predict(setting, True)
        
        torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            arg.model_id,
            arg.model,
            arg.data,
            arg.features,
            arg.seq_len,
            arg.label_len,
            arg.pred_len,
            arg.d_model,
            arg.n_heads,
            arg.e_layers,
            arg.d_layers,
            arg.d_ff,
            arg.factor,
            arg.embed,
            arg.distil,
            arg.des,
            arg.class_strategy, ii)
        
    exp = Exp(arg)  # set experiments
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1)
    torch.cuda.empty_cache()

In [ ]:
# مستقیم بریم برای پیشبینی چون همه چیز در روند ترین مشخص شده

exp.predict(setting,True)

In [ ]:
import os
predicted_path = os.path.join('results', setting, 'Preds real_prediction.npy')

In [ ]:
import os
predicted_path = os.path.join('results', setting, 'Preds real_prediction.npy')
predicted_values = np.load(predicted_path)
predicted_close = predicted_values[0,:,-1].reshape(-1,1)
predicted_close
#This is the last 3 days of the pred dataset :
import pandas as pd
df = pd.read_csv('input/test/data-pred.csv')
df['Close'].tail(4)
import joblib
scaler = joblib.load('input/test/scaler.pkl')
iversed_close = scaler.inverse_transform(predicted_close)
[round(any_) for any_ in iversed_close.reshape(-1).tolist()]
# مدل در هنگام پیش بینی ۱ ردیف آخر را فقط دریافت می کند
# از آنجا که ما با آرگومان مکس ردیف یوز ، برای او مشخص کردیم که ۳ تای آخر را دست نزند
# پس چهارمین مانده به آخر را گرفته است
#برای اینکه از اسکیل مطمئن باشیم آن را بر میگردانیم تا یکسان باشند

scaler.inverse_transform(exp.batch_y[0,:,-1].reshape(-1,1))